In [52]:
import requests
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import time
import pandas as pd
import json

In [11]:
#need to install the chromedriver.exe in the Anaconda3/scripts folder or you will get a path error
driver = webdriver.Chrome()

#fetch url with selenium using chrome looping through to page 18, after that there are no school ratings
school_name_list = []
school_street_list = []
school_zip_list = []
school_city_list = []
school_rating_list = []
page_num = 1

while page_num < 19:
#setting url to get school ratings table that we will scrape.  Will iterate using page_num
    url = f"https://www.greatschools.org/search/search.page?gradeLevels%5B%5D=e&q=houston%20tx&sort=rating&view=table&page={page_num}"
    driver.get(url)
    
#using sleep to ensure page is fully loaded each loop before attempting to scrape    
    time.sleep(8)

#returning selenium html and then switching to beautiful soup
    innerHTML = driver.execute_script("return document.body.innerHTML")
    
    soup = bs(innerHTML,'lxml')
    
#limiting to 25 because the page is duplicating the html then hiding one set.  Was getting 50 out of 25 results

    school_name = soup.find_all('a', class_ ='name',limit=25)
    school_street_zip = soup.find_all('div', class_ ='address',limit=25)
    school_rating = soup.find_all('div', class_ ='circle-rating--small',limit=25)
    
#looping through the above to get text from each element and append to lists

    for name in school_name:
        name = name.text
        school_name_list.append(name)
        
    for street in school_street_zip:
        street = street.text
        school_street_list.append(street[:-20])
        
    for zipcode in school_street_zip:
        zipcode = zipcode.text
        school_zip_list.append(zipcode[-5:])
    
    for city in school_street_zip:
        city = city.text
        school_city_list.append(city[-18:-11])
        
    for rating in school_rating:
        rating = rating.text
        school_rating_list.append(rating[:-3])
    
    page_num += 1


In [28]:
# print len to check if scrape worked properly
print(len(school_city_list))
#,school_street_list,school_zip_list,school_rating_list))

450


In [221]:
schools_dict = {'School':pd.Series(school_name_list),'Street':pd.Series(school_street_list),'Zip':pd.Series(school_zip_list),'City':pd.Series(school_city_list),'Rating':pd.Series(school_rating_list).astype(int)}
schools_df = pd.DataFrame(schools_dict)
#dropping last 6 rows because they did not have ratings
schools_df = schools_df[:-6]
#remove where city != Houston (ie school has Houston in name but not address)
schools_df = schools_df[schools_df.City =='Houston']
schools_df.tail()
#quick visualization of data
#schools_df_plot = schools_df.groupby('Zip').mean()
#schools_df_plot.hist()

,School,Street,Zip,City,Rating
439,Cook Jr Elementary School,7115 Lockwood Drive,77016,Houston,2
440,Admin Services,4250 Cook Road,77072,Houston,1
441,Sneed Elementary School,9855 Pagewood Lane,77042,Houston,1
442,Highland Hts Elementary School,865 Paul Quinn Street,77091,Houston,1
443,Pro-Vision Middle,4590 Wilmington Street,77051,Houston,1


In [250]:
###BLOCK COMMENTED OUT BECAUSE IT TAKES APPROX 30 MINUTES TO RUN###
#Change format of addresses so we can plug into url
school_street_list_new = schools_df.Street.to_list()

formatted_address = []
for i in school_street_list_new:
    formatted = i.replace(' ','+')
    formatted_address.append(formatted)

count=0
fips_list = []

for i in formatted_address:
    street_url = formatted_address[count]
    fips_url=f"https://geocoding.geo.census.gov/geocoder/geographies/address?street={street_url}&city=Houston&state=Tx&benchmark=Public_AR_Census2010&vintage=Census2010_Census2010&layers=14&format=json"
    fips_req = requests.get(fips_url)
    time.sleep(1)
    fips_json = json.loads(fips_req.text)
    
    try:
        fips_list.append([fips_json][0]['result']['addressMatches'][0]['geographies']['Census Blocks'][0]['GEOID'][:-4])
    except Exception:
        fips_list.append('null')
    count +=1
    

#print (json.dumps(fips_json, indent=4))

print(fips_list)



['48201430600', '48201430400', '48201450100', '48201450200', '48201451601', '48201411400', '48201412400', 'null', 'null', '48201520700', '48201421300', '48201432001', '48201332900', 'null', '48201432600', '48201340203', '48201430600', '48201430300', '48201510200', '48201511302', '48201420700', '48201531100', '48201320200', '48201422000', '48201412200', '48201413000', '48201410200', '48201431501', 'null', '48201332900', '48201421300', '48201421300', '48201421102', '48201552400', '48201350100', '48201334001', '48201532300', '48201541201', '48201540100', '48201220600', '48201221200', '48201311100', '48201510400', '48201421700', '48201310300', '48201420800', '48201422701', '48201433202', '48201221300', 'null', '48201541602', 'null', '48201333800', '48201311300', '48201541300', '48201432001', '48201221600', 'null', '48201451401', '48201534203', '48201313400', '48201340400', '48201540100', 'null', '48201212300', '48201452500', '48201451902', '48201340800', 'null', '48201520700', 'null', '482

In [245]:
# fips_list = pd.Series(fips_list)
# schools_df['FIPS'] = fips_list
# schools_df.shape
# # #need to get new list of addresses from the schools_df and use that to get FIPS.  otherwise won't match.


(423, 6)

KeyError: 'FIPS'

KeyError: "['FIPS'] not found in axis"

0      48201430600
1      48201430400
2      48201450100
3      48201450200
4      48201451601
5      48201411400
6      48201412400
7             null
8             null
9      48201520700
10     48201421300
11     48201432001
12     48201332900
13            null
14     48201432600
15     48201340203
16     48201430600
17     48201430300
18     48201510200
19     48201511302
20     48201420700
21     48201531100
22     48201320200
23     48201422000
24     48201412200
25     48201413000
26     48201410200
27     48201431501
28            null
29     48201332900
          ...     
393    48201433600
394    48201522402
395    48201521100
396    48201522302
397    48201520400
398    48201550601
399    48157670101
400    48201432600
401    48201421202
402    48201451300
403    48201422800
404    48201211400
405           null
406    48201331601
407    48201211100
408           null
409    48201332200
410    48201211500
411    48201531600
412    48201530800
413    48201210500
414    48201